<a href="https://colab.research.google.com/github/PosimAndrzej/20475_SI_Lab/blob/main/SI_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [5]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mninst_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [6]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mninst_test.map(scale)

In [7]:
BUFFER_SIZE = 1000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [8]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape, validation_targets.shape)

(6000, 28, 28, 1) (6000,)


In [9]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
NUM_EPOCHS = 30
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data,
          epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 1
          )

Epoch 1/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.7899 - loss: 0.7288 - val_accuracy: 0.9347 - val_loss: 0.2273
Epoch 2/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9433 - loss: 0.1936 - val_accuracy: 0.9508 - val_loss: 0.1648
Epoch 3/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9611 - loss: 0.1371 - val_accuracy: 0.9558 - val_loss: 0.1439
Epoch 4/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9658 - loss: 0.1129 - val_accuracy: 0.9613 - val_loss: 0.1273
Epoch 5/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9721 - loss: 0.0938 - val_accuracy: 0.9622 - val_loss: 0.1183
Epoch 6/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9757 - loss: 0.0835 - val_accuracy: 0.9602 - val_loss: 0.1257
Epoch 7/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9787 - loss: 0.0716 - val_accuracy: 0.9635 - val_loss: 0.1138
Epoch 8/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9808 - loss: 0.0645 - val_accura

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 856ms/step - accuracy: 0.9713 - loss: 0.0992
Test loss: 0.10. Test accuracy: 97.13%


In [38]:
image = cv2.imread('cyfra.png', cv2.IMREAD_GRAYSCALE)


image_resized = cv2.resize(image, (28, 28))
image_normalized = image_resized / 255.0
image_reshaped = image_normalized.reshape(1, 28, 28, 1)


prediction = model.predict(image_reshaped)
predicted_digit = np.argmax(prediction)


print(f'Przewidziana cyfra: {predicted_digit}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Przewidziana cyfra: 4
